'''
OBJECTIVES:
1. Build WRS system
2. Build Structural BMP Solution evaluator
3. Identify minimum BMP solution front for:
   individual facilities
   facilities w/in departments
   facilities w/in city
'''

In [1]:
'''
Define basic SQLAlchemy items:
    declarative base object
    connection object
    session object
    DB tables
'''
#SQLAlchemy library items:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String
from sqlalchemy import update, insert
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship #http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html#relationship-patterns
from sqlalchemy import inspect

from SQLA_Base import Base #module containing declarative_base
from SQLA_conn_man import session, engine #module handling db and connection creation 

#Table definitions as SQLA classes: 
from SQLA_DB_base_bmp_feasibility_test_definitions import Base_BMP_Feasibility_Test_Definitions
from SQLA_DB_base_bmps import Base_BMPs
from SQLA_DB_expressions import Expressions
from SQLA_DB_facility_chars import Facility_Chars
from SQLA_DB_facility_monthly_rain import Facility_Monthly_Rain
from SQLA_DB_facility_risks import Facility_Risks
from SQLA_DB_facility_type_has_nel import Facility_Type_Has_NEL
from SQLA_DB_facility_types import Facility_Types
from SQLA_DB_feasibility_test_questions import Feasibility_Test_Questions
from SQLA_DB_pollutant_removal_rates import Pollutant_Removal_Rates
from SQLA_DB_wrs_pollutant_risks import WRS_Pollutant_Risks
Base.metadata.create_all(engine, checkfirst=True) #create SQLA classes

'''
Dictionary of "SQLAlchemy where clause lambda functions" that importCSV uses to test record uniqueness.
used as the where clause in sqlalchemy queries, updates and deletes 
Form:
    TableName:Lambda Function
    
    TableName is the table name we want to define uniqueness test for
    Lambda Function can take on any form but must be made to evaluate the CSV row passed as a dictionary (CSVRowDict in this explanation):
        CSVRowDict: {FieldName:CSVColValue, DBTableFieldName:CSVColValue...} 
            Where: DBTableFieldName is the name of the field associated with the value at CSVColValue on the current row
               CSVColValue: a value in the CSV's current row+column corresponding to the DBTableFieldName 
        *this assumes that field names are unique across table. if not, then method fails (maybe need to extend method?)
        
e.g.: lambda myRowVal: Base.metadata.tables['people'].c['name'] == CSVRowDict['name']
        using lambda function in query will search for CSVRowDict's value for 'name' in the table people, field name 
if table has no record uniqueness requirement, then enter: TableName:False
'''
unqTests = {
    'facility_chars': lambda CSVRowDict: Base.metadata.tables['facility_chars'].c['Fac_Name'] == CSVRowDict['Fac_Name'],
    'facility_monthly_rain': False, #DB schema does not impose uniqueness on records in this table
    'facility_type_has_nel': False,
    'facility_types': lambda CSVRowDict: Base.metadata.tables['facility_types'].c['Fac_Type'] == CSVRowDict['Fac_Type'],
    'wrs_pollutant_risks': False #DB schema does not impose uniqueness on records in this table
}

import SQLA_main as SQLA_main #import main SQLAlchemy functions



Clearing old DB


In [2]:
'''
Define other custom modules

'''
import expression as Expr
import importSpecial as importSpecial #special import functions are defined here
import importCSV as importCSV #generic CSV importer ****IMPORTANT NOTE: function assumes csv in the utf-8-sig file format. weird things happen if its not in this format!!!


In [3]:
#import feasibillity questions, build feasibility expressions
importSpecial.importFeasibilityQuestionsCSV('Input_Files\\feasibility_test_questions.csv') 

#import base bmp information including:
  #1. imports definitions for cip costs, o&m costs, and BMP sizing to the expressions table
  #2. imports pollutant removal rates into pollutant_removal_rates table
  #3. creates a record in the base_bmps table using (1) and (2)
  #4. feasibility tests
importSpecial.importBaseBMPsCSV('Input_Files\\bmp_lego_piece.csv') 

#IMPORT BASIC FACILITY CHARS:
print ('\nImporting facility characteristics:')
importCSV.importCSV('Input_Files\\facility_chars.csv', unqTests)

#IMPORT FACILITY RAINFALL EXTRACTED FROM http://rainfall.geography.hawaii.edu/downloads.html
print ('\nImporting Facility Rainfall Data:')
importCSV.importCSV('Input_Files\\FacilityRainfallData.csv', unqTests)

#IMPORT EFFLUENT LIMITS FOR FACILITY TYPES: (either by Priority Based Plan, Table 3 or as City operational assignment)


print ('\nImporting Facility Type Has Effluent Limits:') #import into wrs_pollutant_risks table
importCSV.importCSV('Input_Files\\nel_exists_facility_types.csv', unqTests)

session.commit()

Reading csv for import to Feasibility Questions

Reading csv record: Feas-1
Adding to variable dictionary: OFFSITE_SD_Exist

Reading csv record: Feas-2
Adding to variable dictionary: GW_Risk

Reading csv record: Feas-3
Adding to variable dictionary: GW_Risk

Reading csv record: Feas-4
Adding to variable dictionary: Soil_Type

Reading csv record: Feas-5
Adding to variable dictionary: Soil_Type

Reading csv record: Feas-6
Adding to variable dictionary: Soil_Type

Reading csv record: Feas-7
Adding to variable dictionary: Count_CB

Reading csv record: Feas-8
Adding to variable dictionary: Runoff_Type

Reading csv record: Feas-9
Adding to variable dictionary: TFMR_Exist

Reading csv record: Feas-10
Adding to variable dictionary: DS_SS_Exist

Reading csv record: Feas-11
Adding to variable dictionary: Fac_Slope

Reading csv record: Feas-12
Adding to variable dictionary: Can_Add_SD

Reading csv record: Feas-13
Adding to variable dictionary: Pave_Area
Adding to variable dictionary: BMP_Size(bas

{'Fac_Name': 'Pearl City Fire Station'}
{'Fac_Name': 'Kapolei Police Station'}
{'Fac_Name': 'Kalihi Police Station'}
{'Fac_Name': 'Kahuku Police Station'}
{'Fac_Name': 'Mokulele Fire Station and Training Center'}
{'Fac_Name': 'Kaneohe Police Station'}
{'Fac_Name': 'Kaneohe Fire Station'}
{'Fac_Name': 'Alapai Police Station'}
{'Fac_Name': 'Kailua Police Station'}
{'Fac_Name': 'Mililani-Mauka Fire Station'}
{'Fac_Name': 'Kalihi-Uka Fire Station'}
{'Fac_Name': 'Nuuanu Fire Station'}
{'Fac_Name': 'Waiau Fire Station'}
{'Fac_Name': 'Kapolei Fire Station'}
{'Fac_Name': 'Pearl City Police Station'}
{'Fac_Name': 'Central Fire Station'}
{'Fac_Name': 'Olomana Fire Station'}
{'Fac_Name': 'Waipahu Vehicle Maintenance Shop'}
{'Fac_Name': 'Sunset Beach Fire Station'}
{'Fac_Name': 'Waikele Fire Station'}
{'Fac_Name': 'Wahiawa Fire Station'}
{'Fac_Name': 'Manoa Fire Station'}
{'Fac_Name': 'Wahiawa Police Station'}
{'Fac_Name': 'Makiki Fire Station'}
{'Fac_Name': 'Waianae Police Station'}
{'Fac_Name': 

{'Fac_Name': 'Kaneohe Bay #3 Pump Station'}
{'Fac_Name': 'Kaneohe Bay #5 Pump Station'}
{'Fac_Name': 'Lualualei Pump Station'}
{'Fac_Name': 'Makakilo Pump Station'}
{'Fac_Name': 'Nanakuli Pump Station'}
{'Fac_Name': 'Paiko Drive Pump Station'}
{'Fac_Name': 'Public Baths Pump Station'}
{'Fac_Name': 'West Beach #1 Pump Station'}
{'Fac_Name': 'West Beach #2 Pump Station'}
{'Fac_Name': 'West Loch Estates Pump Station'}
{'Fac_Name': 'Kapolei Business Park Wastewater Pump Station'}
{'Fac_Name': 'Kuliouou Pump Station'}
{'Fac_Name': 'Waipio Peninsula Soccer Complex (including Maintenance Baseyard)'}
{'Fac_Name': 'Honolulu Zoo'}
{'Fac_Name': 'Airport Vehicle Maintenance Yard'}
{'Fac_Name': 'Kaneohe Wastewater Preliminary Treatment Facility'}
{'Fac_Name': 'Neal Blaisdell Center'}
{'Fac_Name': 'Nuuanu Nursery'}
{'Fac_Name': 'Ahuimanu Pre-Treatment Facility'}
{'Fac_Name': 'Umi Street (AES) Fueling Facility'}
{'Fac_Name': 'Waipahu Tree Farm'}
{'Fac_Name': 'Auahi Coning Yard'}
{'Fac_Name': 'Kaneohe

In [4]:
session.close()
engine.dispose()